In [23]:
import numpy as np
import pandas as pd
import xlsxwriter
import time
import h5toDF
import xlautofit
import math
from summary_functions import *

In [24]:
#################################### WHERE ARE YOU RUNNING? ####################################
model_dir = 'Z:/Stefan/Soundcast_feb_twg/'


### OTHER PATHS. FOR A TYPICAL RUN, YOU DON'T HAVE TO CHANGE THESE ######################################
h5_results_file = 'outputs/daysim_outputs.h5'
h5_results_name = 'DaysimOutputs'
h5_comparison_file = 'scripts/summarize/survey.h5'
h5_comparison_name = 'Survey'
guidefile = 'scripts/summarize/inputs/calibration/CatVarDict.xlsx'
districtfile = 'scripts/summarize/inputs/calibration/TAZ_TAD_County.csv'
report_output_location = 'outputs'

parcel_decay_file = 'inputs/buffered_parcels.dat'



h5_results_file = model_dir + h5_results_file
h5_comparison_file =  model_dir + h5_comparison_file
guidefile = model_dir + guidefile
districtfile = model_dir + districtfile
report_output_location = 'outputs'


In [25]:
#READ IN YOUR DATA
data1 = h5toDF.convert(h5_results_file,guidefile,h5_results_name)
data2 = h5toDF.convert(h5_comparison_file,guidefile, h5_comparison_name)

---Begin DaysimOutputs conversion---
Guide import complete
Guide converted to dictionary in 0.0 seconds
Household File import/recode complete in 1.4 seconds
HouseholdDay File import/recode complete in 0.5 seconds
Person File import/recode complete in 5.1 seconds
PersonDay File import/recode complete in 2.1 seconds
Tour File import/recode complete in 7.0 seconds
Trip File import/recode complete in 20.1 seconds
---DaysimOutputs import/recode complete in 36.1 seconds---
---Begin Survey conversion---
Guide import complete
Guide converted to dictionary in 0.0 seconds
Household File import/recode complete in 0.0 seconds
HouseholdDay File import/recode complete in 0.0 seconds
Person File import/recode complete in 0.1 seconds
PersonDay File import/recode complete in 0.0 seconds
Tour File import/recode complete in 0.1 seconds
Trip File import/recode complete in 0.1 seconds
---Survey import/recode complete in 0.4 seconds---


In [26]:
trip_variables = ['otaz', 'dtaz', 'travtime', 'travcost', 'travdist', 'pno', 'mode', 'tour_id', 'opcl', 'dpcl', 'dorp', 'dpurp', 'opurp', 'trexpfac']
hh_variables = ['hhno', 'hhincome', 'hhvehs', 'hhtaz', 'hhexpfac']
person_variables = ['hhno', 'pno', 'pagey', 'pgend', 'id']

In [27]:
def get_variables_trips_model(output_df,trip_variables, hh_variables, person_variables):
    trip_data = output_df['Trip'][trip_variables]
    hh_data = output_df['Household'][hh_variables]
    person_data = output_df['Person'][person_variables]
    tour_data = output_df['Tour'][['hhno', 'pno', 'id']]
    tour_data.rename(columns = {'id': 'tour_id'}, inplace = True)

    merge_hh_person = pd.merge(hh_data, person_data, 'inner', on = 'hhno')
    merge_hh_person.reset_index()
    tour_data.reset_index()
    merge_hh_tour = pd.merge(merge_hh_person, tour_data, 'inner', on =('hhno', 'pno'))
    merge_trip_hh = pd.merge(merge_hh_tour, trip_data, 'outer', on= 'tour_id')
    return merge_trip_hh  

In [28]:
def get_variables_trips_survey(output_df,trip_variables, hh_variables):
    trip_data = output_df['Trip'][trip_variables]
    hh_data = output_df['Household'][hh_variables]

    merge_trip_hh = pd.merge(trip_data, hh_data, on = 'hhno')
    return merge_trip_hh

In [29]:
trips_model = get_variables_trips_model(data1, trip_variables, hh_variables, person_variables)

In [30]:
trips_survey = get_variables_trips_model(data2, trip_variables, hh_variables, person_variables)

In [31]:
rgc_file = pd.read_csv(model_dir+'scripts/summarize/inputs/rgc_taz.csv')

In [32]:
rgc_file.head()

,taz,center,lat_1,lon_1
0,3317,Puyallup South Hill,47.156235,-122.286753
1,3051,Lakewood,47.161986,-122.510059
2,3314,Puyallup South Hill,47.156235,-122.286753
3,3315,Puyallup South Hill,47.156235,-122.286753
4,3046,Lakewood,47.161986,-122.510059


In [33]:
trips_rgc = pd.merge(trips_model, rgc_file, left_on = 'dtaz', right_on='taz')

In [34]:
trips_rgc_mode =trips_rgc.groupby(['mode', 'center']).count().reset_index()

In [35]:
trips_rgc_mode

,mode,center,hhno,hhincome,hhvehs,hhtaz,hhexpfac,pno_x,pagey,pgend,...,pno_y,opcl,dpcl,dorp,dpurp,opurp,trexpfac,taz,lat_1,lon_1
0,Bike,Auburn,184,184,184,184,184,184,184,184,...,184,184,184,184,184,184,184,184,184,184
1,Bike,Bellevue,1797,1797,1797,1797,1797,1797,1797,1797,...,1797,1797,1797,1797,1797,1797,1797,1797,1797,1797
2,Bike,Bothell Canyon Park,405,405,405,405,405,405,405,405,...,405,405,405,405,405,405,405,405,405,405
3,Bike,Bremerton,308,308,308,308,308,308,308,308,...,308,308,308,308,308,308,308,308,308,308
4,Bike,Burien,361,361,361,361,361,361,361,361,...,361,361,361,361,361,361,361,361,361,361
5,Bike,Everett,647,647,647,647,647,647,647,647,...,647,647,647,647,647,647,647,647,647,647
6,Bike,Federal Way,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70
7,Bike,Issaquah,168,168,168,168,168,168,168,168,...,168,168,168,168,168,168,168,168,168,168
8,Bike,Kent,521,521,521,521,521,521,521,521,...,521,521,521,521,521,521,521,521,521,521
9,Bike,Kirkland Totem Lake,868,868,868,868,868,868,868,868,...,868,868,868,868,868,868,868,868,868,868


In [36]:
trips_survey_rgc =pd.merge(trips_survey, rgc_file, left_on = 'dtaz', right_on='taz')

In [37]:
trips_survey_rgc_mode =trips_survey_rgc.groupby(['mode', 'center']).sum().reset_index()

In [38]:
trips_survey_model = pd.merge(trips_rgc_mode, trips_survey_rgc_mode, on = ['mode','center'])

In [39]:
trips_survey_model.to_clipboard()